**Task 04: Graph querying**

In [2]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/daniel-dona/Curso2018-2019/master/Assignment3/"

     |████████████████████████████████| 235kB 5.1MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 


In [3]:
from rdflib import Graph, Namespace, Literal
g = Graph()
g.parse(github_storage+"/resources/example3.rdf", format="xml")

<Graph identifier=N1a5e9cc13cd84710a4f9ff0d7fa8bc7e (<class 'rdflib.graph.Graph'>)>

Listamos todos los recursos que contienen la propiedad VCARD:FN

In [ ]:
from rdflib.plugins.sparql import prepareQuery


VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")


q1 = prepareQuery('''
  SELECT 
    ?Subject
  WHERE { 
    ?Subject vcard:FN ?FullName. 
  }
  ''',
  initNs = { "vcard": VCARD}
)


for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#MattJones'),)
(rdflib.term.URIRef('http://somewhere#SarahJones'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


Repetimos la anterior consulta, pero pidiendo ahora además los nombres completos de los sujetos

In [ ]:
q2 = prepareQuery('''
  SELECT 
    ?Subject ?FullName
	WHERE { 
    ?Subject vcard:FN ?FullName.
  } 
  ''',
  initNs = { "vcard": VCARD}
)

for r in g.query(q2):
  print(r)

(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.Literal('John Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#MattJones'), rdflib.term.Literal('Matt Jones', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SarahJones'), rdflib.term.Literal('Sarah Jones', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JaneSmith'), rdflib.term.Literal('Jane Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))


Obtenemos todos los recursos que contienen "Smith" como nombre de familia

In [ ]:
from rdflib import XSD

q3 = prepareQuery('''
  SELECT 
    ?Subject ?x
	WHERE { 
    ?Subject vcard:Family ?Family.
  } 
  ''',
  initNs = { "vcard": VCARD}
)

for r in g.query(q3, initBindings = {'?Family' : Literal('Smith', datatype=XSD.string)}):
  print(r)

(rdflib.term.URIRef('http://somewhere#JaneSmith'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)


Obtenemos todos los elementos que contienen un email asociado

In [ ]:
from rdflib import FOAF

q4 = prepareQuery('''
  SELECT 
    ?Subject ?Email
	WHERE { 
    ?Subject foaf:email ?Email.
  } 
  ''',
  initNs = { "foaf": FOAF}
)

for r in g.query(q4):
  print(r)

(rdflib.term.URIRef('http://somewhere#JaneSmith'), rdflib.term.Literal('jSmith@somewhere.com', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SarahJones'), rdflib.term.Literal('sJones@somewhere.com', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))


Consultamos todos los que conocen (FOAF:knows) a "Jane Smith" y obtenemos sus nombres de pila (VCARD:Given)

In [ ]:
q5 = prepareQuery('''
  SELECT 
    ?Subject ?Given
  WHERE {
    ?Subject foaf:knows ?JaneSmith.
		?JaneSmith vcard:FN ?JaneSmithFullName.
		?Subject vcard:Given ?Given.
  }  
  ''',
  initNs = { "foaf": FOAF, "vcard": VCARD, "xsd":XSD}
)

for r in g.query(q5, initBindings = {'?JaneSmithFullName' : Literal('Jane Smith', datatype=XSD.string)}):
  print(r)

(rdflib.term.URIRef('http://somewhere#MattJones'), rdflib.term.Literal('Matt', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.Literal('John', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
